In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from src.robot_reboot.dataset_factory import RobotRebootDataSetFactory

In [2]:
cnn = tf.keras.models.load_model('model/model_0')
cnn.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 31, 31, 9)]  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 31, 31, 64)   9280        input[0][0]                      
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 15, 15, 64)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 15, 15, 64)   65600       max_pooling2d_6[0][0]            
____________________________________________________________________________________________

In [3]:
ds_factory = RobotRebootDataSetFactory(31, cnn, max_depth=20, seed=26, playouts=50)

In [6]:
rows, cols, layers = 31, 31, 9
robots = int((layers - 1)/2)
p_columns = [f'p_{i}' for i in range(robots * 4)]
maze_columns = [f'm_{i}' for i in range(rows * cols *layers)]
columns = ['v'] + p_columns + maze_columns

In [7]:
def create(ds_factory, max_movements, i):
    print(f'Creating sample: {i}')
    v, p, s = ds_factory.create(locate_robot_close_goal=True, max_movements=max_movements)
    p_and_maze = np.concatenate((p, s.reshape(1, rows * cols * layers)), axis=None)
    data = np.concatenate((np.array([v]), p_and_maze), axis=None).reshape(1, 8666)
    df = pd.DataFrame(data = data, columns = columns)
    print(f'Saving data/ds_{i}.csv')
    df.to_csv(f'data/ds_{i}.csv', header=False, index=False)

In [9]:
from threading import Thread

np.random.seed(26)
threads = list()
for i in range(0, 2):
    if i% 20 ==0 and i!=0:
        print(f'Created {i}/600')
        [t.join() for t in threads]
        threads = list()
    max_movements = np.random.randint(1, 6)
    thread = Thread(target = create, args=(ds_factory, max_movements, i))
    thread.start()
    threads.append(thread)
[t.join() for t in threads]

Creating sample: 0
Creating sample: 1
Saving data/ds_0.csv
Saving data/ds_1.csv


[None, None]

In [17]:
import glob

path = r'data'
all_files = glob.glob(path + "/*.csv")
ds = list()
for filename in all_files:
    df = pd.read_csv(filename, index_col =None, header=0)
    ds.append(df)

In [18]:
df = pd.concat(ds, axis = 0, ignore_index=True)

In [19]:
df.columns

Index(['0.0', '1.0', '0.0.1', '0.0.2', '0.0.3', '0.3', '0.28', '0.28.1', '0.4',
       '0.4.1',
       ...
       '0.0.8604', '0.0.8605', '0.0.8606', '0.0.8607', '0.0.8608', '0.0.8609',
       '0.0.8610', '0.0.8611', '0.0.8612', '0.0.8613'],
      dtype='object', length=8680)